# Chuck Sandbox

In [1]:
# Dependencies
import pandas as pd
import numpy as np


# Save file path to variable
#tennis_pop_2018_csv = "Resources/state_populations_2018.csv"
tennis_raw_csv = "../Resources/tennis_courts.csv"
state_populations_csv = "../Resources/State Populations.csv"
gallup_2018_csv = "../Resources/Gallup2018.csv"
tennis_manually_gathered_csv = "../Resources/tennis_courts_manually_gathered.csv"
consolidated_csv = "../Resources/consolidated_tennis.csv"

#read data into a dataset
tennis_raw_df = pd.read_csv(tennis_raw_csv)
#tennis_raw_df.head(2)

In [2]:
# # Read Gallup with Pandas
gallup_df = pd.read_csv(gallup_2018_csv)
#gallup_df.rename(columns = {'State':'state'}, inplace = True)
gallup_df.head(2)

,State,2018 sample size,Well-Being Index,Well-Being Index Rank,Career,Social,Financial,Community,Physical
0,Hawaii,502,64.6,1,1,1,1,4,4
1,Wyoming,219,64.2,2,3,18,6,1,3


In [3]:
# Read state population data into a dataset
state_population_df = pd.read_csv(state_populations_csv)
state_population_df = state_population_df.loc[state_population_df['State'] != 'District of Columbia']
state_population_df.head(2)


,State,2018 Population,state_abbreviation
0,Alaska,738068,ak
1,Alabama,4888949,al


In [4]:
# state_population_df.describe()

In [5]:
# # Count of States
# state_pop_count = state_population_df.nunique()
# state_pop_count

In [6]:
# manually gathered count of public tennis courts data

# Read manually gathered data into a dataset
tennis_manually_gathered_df = pd.read_csv(tennis_manually_gathered_csv)
tennis_manually_gathered_df.head()

,state_abbreviation,count,source
0,nj,788,manually gathered by searching for count of pu...
1,sc,93,manually gathered by searching for count of pu...
2,al,33,manually gathered by searching for count of pu...
3,la,59,manually gathered by searching for count of pu...
4,ia,75,manually gathered by searching for count of pu...


In [7]:
# Slim down the manually gathered set and to state_abbreviation and count of public court locations
tennis_manually_gathered_reduced_df = tennis_manually_gathered_df[["state_abbreviation", "count"]]
tennis_manually_gathered_reduced_df = tennis_manually_gathered_reduced_df.set_index("state_abbreviation")
tennis_manually_gathered_reduced_df.head()

,count
state_abbreviation,
nj,788
sc,93
al,33
la,59
ia,75


In [8]:
# Raw public public tennis court location data for each state obtained from database
tennis_raw_reduced_df = tennis_raw_df[["state_abbreviation", "type", "count"]]
tennis_raw_reduced_df.head(2)

,state_abbreviation,type,count
0,az,Homeowners Community,1
1,az,Homeowners Community,2


In [9]:
# Filter raw tennis where type = 'Public'
tennis_public_df = tennis_raw_reduced_df.loc[tennis_raw_reduced_df['type'] == "Public"]
tennis_public_df.head()

,state_abbreviation,type,count
9,az,Public,1
11,az,Public,8
12,az,Public,10
16,az,Public,8
18,az,Public,6


In [10]:
# Remove column for type
tennis_public_df.drop(columns="type",inplace=True)

C:\Users\Janssen VR 2\AppData\Local\Temp\ipykernel_22800\1228967714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tennis_public_df.drop(columns="type",inplace=True)


In [11]:
tennis_public_df.head()

,state_abbreviation,count
9,az,1
11,az,8
12,az,10
16,az,8
18,az,6


In [12]:
#New data frame with count of pulblic tennnis courts by state sourced from database
tennis_counts_df = tennis_public_df.groupby(["state_abbreviation"]).sum("count")


In [13]:
tennis_counts_df.head(2)


,count
state_abbreviation,
az,1331
ca,9658


In [14]:
tennis_manually_gathered_reduced_df.head(2)

,count
state_abbreviation,
nj,788
sc,93


In [15]:
# Combine database sourced, and manually sourced public tennis court count data into the same df 
frames = [tennis_counts_df, tennis_manually_gathered_reduced_df]

In [16]:
# Concatinate the frames
public_tennis_court_count_df = pd.concat(frames)

In [17]:
public_tennis_court_count_df.head(2)

,count
state_abbreviation,
az,1331
ca,9658


In [18]:
#Merge tennis court count data with state population data

state_population_df.head(2)

,State,2018 Population,state_abbreviation
0,Alaska,738068,ak
1,Alabama,4888949,al


In [19]:
# Merge of State population and public tennis court tennis court data
# Merge two dataframes using an inner join
merge_state_pop_public_tennis_df = pd.merge(state_population_df, public_tennis_court_count_df, on='state_abbreviation')
merge_state_pop_public_tennis_df.head(2)

,State,2018 Population,state_abbreviation,count
0,Alaska,738068,ak,42
1,Alabama,4888949,al,33


In [20]:
# Add a new calculated column for the public_tennis_court_index
merge_state_pop_public_tennis_df["public_tennis_court_index"] = merge_state_pop_public_tennis_df["count"]/merge_state_pop_public_tennis_df["2018 Population"]*10000
merge_state_pop_public_tennis_df.head(2)


,State,2018 Population,state_abbreviation,count,public_tennis_court_index
0,Alaska,738068,ak,42,0.569053
1,Alabama,4888949,al,33,0.067499


In [21]:
# Merge the state public tennis court and population data with gallup data using state
gallup_df.head(2)

,State,2018 sample size,Well-Being Index,Well-Being Index Rank,Career,Social,Financial,Community,Physical
0,Hawaii,502,64.6,1,1,1,1,4,4
1,Wyoming,219,64.2,2,3,18,6,1,3


In [22]:
# last merge >> Gallup data with the state pop and public tennis court data
# need to rank by 

merge_state_pop_public_tennis_gallup_df = pd.merge(gallup_df, merge_state_pop_public_tennis_df, on='State')

merge_state_pop_public_tennis_gallup_df= merge_state_pop_public_tennis_gallup_df.sort_values(by=['Well-Being Index Rank'])

merge_state_pop_public_tennis_gallup_df.head(2)



,State,2018 sample size,Well-Being Index,Well-Being Index Rank,Career,Social,Financial,Community,Physical,2018 Population,state_abbreviation,count,public_tennis_court_index
0,Hawaii,502,64.6,1,1,1,1,4,4,1426393,hi,91,0.637973
1,Wyoming,219,64.2,2,3,18,6,1,3,573720,wy,10,0.174301


In [23]:
# This is a set where the transformations were done in excel
# # 
consolidated_df = pd.read_csv(consolidated_csv)
consolidated_df= consolidated_df.sort_values(by=['Well-Being Index Rank'])
consolidated_df.head(2)


,State,2018 sample size,Well-Being Index,Well-Being Index Rank,Career,Social,Financial,Community,Physical,2018 Population,state_abbreviation,count_public_court_locations,public_courts_per_10000_people,public_courts_rank
0,Hawaii,502,64.6,1,1,1,1,4,4,1426393,hi,91,0.637973,31
1,Wyoming,219,64.2,2,3,18,6,1,3,573720,wy,10,0.174301,42
